In [136]:
import pandas as pd
import numpy as np

# Visualisation 
import matplotlib.pyplot as plt  
import seaborn as sns  
from sklearn.metrics import accuracy_score, confusion_matrix
import os
from datetime import datetime
import pickle

# Keras for deep learning models
from tensorflow.keras.preprocessing.text import Tokenizer  # For tokenising text data
from tensorflow.keras.preprocessing.sequence import pad_sequences  # For sequence padding to uniform length
from tensorflow.keras.models import Sequential  # For sequential model architecture
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Bidirectional, Dropout  # Layers for model building
from tensorflow.keras.regularizers import l2  # L2 regularisation to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History
from tensorflow.keras.callbacks import CSVLogger

In [137]:
def prepare_data():
    
    # Load the dataset from a CSV file
    data = pd.read_csv('car-reviews.csv')
    
    # Splitting the data into positive and negative reviews based on a predetermined index
    positive_data = data[691:]  
    negative_data = data[:691]  
    
    # Further split positive and negative datasets into training and testing subsets
    train_positive = positive_data[:553]
    test_positive = positive_data[553:] 
    train_negative = negative_data[:553]
    test_negative = negative_data[553:]  
    
    # Concatenate the positive and negative training data, and do the same for testing data
    training_data = pd.concat([train_positive, train_negative]).reset_index(drop=True)
    testing_data = pd.concat([test_positive, test_negative]).reset_index(drop=True)
    
    # Convert the 'Sentiment' column into a binary format: 1 for positive, 0 for negative
    training_data['Sentiment'] = np.where(training_data['Sentiment'] == 'Pos', 1, 0)
    testing_data['Sentiment'] = np.where(testing_data['Sentiment'] == 'Pos', 1, 0)

    # Extract reviews (as text) and their corresponding labels for training and testing
    training_texts = list(training_data['Review'])
    training_labels = list(training_data['Sentiment'])
    test_texts = list(testing_data['Review'])
    test_labels = list(testing_data['Sentiment'])

    # Return the processed text and labels for training and testing
    return list((training_texts, training_labels, test_texts, test_labels))

In [142]:
class LSTM_Classifier():

    model = None
    history = None

    def __init__(self, data, load_model_toggle, early_stopping_patience, validation_split, tokeniser_word_count, pad_sequences_maxlen, embedding_input_dim, 
                 embedding_output_dim, lstm_unit, dropout_rate, reg_strength, epochs, batch_size, dense_output_activation_function, model_optimisation, model_loss_function):

        self.data = data
        self.load_model_toggle = load_model_toggle
        self.early_stopping_patience = early_stopping_patience
        self.validation_split = validation_split

        self.tokeniser_word_count = tokeniser_word_count
        self.pad_sequences_maxlen = pad_sequences_maxlen

        self.embedding_input_dim = embedding_input_dim
        self.embedding_output_dim = embedding_output_dim
        self.lstm_unit = lstm_unit
        self.dropout_rate = dropout_rate
        self.reg_strength = reg_strength
        
        self.epochs = epochs
        self.batch_size = batch_size
        self.dense_output_activation_function = dense_output_activation_function
        self.model_optimisation = model_optimisation
        self.model_loss_function = model_loss_function 


    

    
    def tokenisation(self):
        
        # Initialise the tokenizer with a maximum number of words to keep, based on word frequency
        tokenizer = Tokenizer(num_words=self.tokeniser_word_count)  # Setting the size of the vocabulary to the top 10,000 words
        tokenizer.fit_on_texts(self.data[0])  # Updates internal vocabulary based on the list of texts
        
        # Convert the list of texts to a sequence of integers
        train_sequences = tokenizer.texts_to_sequences(self.data[0])  # Transforms each text in training_texts to a sequence of integers
        self.data[0] = pad_sequences(train_sequences, maxlen=self.pad_sequences_maxlen)  # Ensures all sequences in a list have the same length by padding/truncating
        self.data[1] = np.array(self.data[1])  
        
        # Repeat the tokenisation and padding process for the testing set
        test_sequences = tokenizer.texts_to_sequences(self.data[2]) 
        self.data[2] = pad_sequences(test_sequences, maxlen=self.pad_sequences_maxlen) 
        self.data[3] = np.array(self.data[3])  


    
    

    def LTSM_model(self):
    
        self.model = Sequential([
            
            # Input layer specifies the shape of input data = pad_sequences_maxlen
            Input(shape=(self.pad_sequences_maxlen,)),
            
            # Embedding layer to turn positive integers (indexes) into dense vectors of fixed size, 10000 is the size of the vocabulary
            Embedding(input_dim=self.embedding_input_dim, output_dim=self.embedding_output_dim),
            
            # LSTM layer with specified units and dropout for regularization
            # Wrap the LSTM layer with a Bidirectional layer
            Bidirectional(LSTM(self.lstm_unit, dropout=self.dropout_rate, recurrent_dropout=self.dropout_rate)),
            
            # Dense output layer with sigmoid activation for binary classification
            Dense(1, activation=self.dense_output_activation_function, kernel_regularizer=l2(self.reg_strength))
        ])
    
        if self.load_model == True:
            model = self.load_model()
        else:
            model = self.compile_model()

        self.evaluate_model()



    
    
    def evaluate_model(self):

        # Predict probabilities on the test set and convert to class labels (1 or 0) based on a 0.5 threshold
        test_probabilities = self.model.predict(self.data[2])
        test_predictions = (test_probabilities > 0.5).astype("int32").flatten()
        
        # Generating the confusion matrix from test labels and predictions
        conf_matrix = confusion_matrix(self.data[3], test_predictions)
        conf_matrix_df = pd.DataFrame(conf_matrix, index=["Actual Negative", "Actual Positive"], columns=["Predicted Negative", "Predicted Positive"])
        print('\n')
        display(conf_matrix_df)
    
        accuracy = accuracy_score(self.data[3], test_predictions)
        print('\n')
        print('Accuracy: ', round(accuracy, 3))
        print('\n')
    
        # Call the model evaluation function to plot training history
        self.visualise_accuracy()



    

    def compile_model(self):
    
        # Compile the model with Adam optimizer and binary crossentropy loss, tracking accuracy
        self.model.compile(optimizer=self.model_optimisation, loss=self.model_loss_function, metrics=['accuracy'])
    
        # Save the model params and weight with the lowest validation loss and add early stopping if validation loss does not decrease for 2 epochs 
        checkpoint_filepath = os.getcwd() + f'/LSTM_Models/weights/LSTM_Model_{datetime.now().strftime("%Y%m%d_%H%M%S")}.keras'
        model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_loss', mode='min', save_best_only=True)
        callbacks = [EarlyStopping(patience=self.early_stopping_patience), model_checkpoint_callback]
        
        # Train the model on the training data, with a validation split to monitor overfitting
        self.history = self.model.fit(self.data[0], self.data[1], batch_size=self.batch_size, epochs=self.epochs, validation_split=self.validation_split, callbacks=callbacks)

        # Save the history file
        with open(f'LSTM_Models/histories/LSTM_History_{unique_id}.pkl', 'wb') as history_file:
            pickle.dump(self.history, history_file)

    

    def load_model(self):
    
        model.load_weights('/LSTM_Models/weights/LSTM_Model_20240404_141225.keras')
        self.history = self.model.fit(self.data[0], self.data[1], batch_size=self.batch_size, epochs=self.epochs, validation_split=self.validation_split, callbacks=callbacks)
        

    
    def visualise_model_params(self):
        
        # Example: Visualize the weights of the first Dense layer
        dense_layer_weights = model.layers[2].get_weights()[0]  # 0 for weights, 1 for biases
        plt.hist(dense_layer_weights.flatten())
        plt.xlabel('Weight values')
        plt.ylabel('Frequency')
        plt.title('Distribution of Weight Values')
        plt.show()


    

    def visualise_accuracy(self):
        
        # Accessing training and validation accuracy and loss from the history object
        training_accuracy = self.history.history['accuracy']
        validation_accuracy = self.history.history['val_accuracy']
        training_loss = self.history.history['loss']
        validation_loss = self.history.history['val_loss']
        epochs = range(1, len(training_accuracy) + 1)
        
        # Use a dark background style for plots
        with plt.style.context('dark_background'):
            
            plt.figure(figsize=(10, 4))
            
            # Plotting training and validation accuracy
            plt.subplot(1, 2, 1)
            plt.plot(epochs, training_accuracy, label='Training Accuracy', marker='o', color='lime')
            plt.plot(epochs, validation_accuracy, label='Validation Accuracy', marker='o', color='cyan')
            plt.title('Training and Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            
            # Plotting training and validation loss
            plt.subplot(1, 2, 2)
            plt.plot(epochs, training_loss, label='Training Loss', marker='o', color='lime')
            plt.plot(epochs, validation_loss, label='Validation Loss', marker='o', color='cyan')
            plt.title('Training and Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            plt.tight_layout()
            plt.show()



    
    def run(self):
        self.tokenisation()
        self.LTSM_model()

In [ ]:
data = prepare_data()

lstm_classifier = LSTM_Classifier(data, 
                                  load_model_toggle = True,
                                  early_stopping_patience = 2,
                                  validation_split = 0.2,
                                  tokeniser_word_count = 10000,
                                  pad_sequences_maxlen = 100,
                                  embedding_input_dim = 10000,
                                  embedding_output_dim = 62,
                                  lstm_unit = 48,
                                  dropout_rate = 0.2,
                                  reg_strength = 0.3,
                                  epochs = 20,
                                  batch_size = 32,
                                  dense_output_activation_function = 'sigmoid',
                                  model_optimisation = 'adam',
                                  model_loss_function = 'binary_crossentropy',
                                 )

lstm_classifier.run()

Epoch 1/20
